## Step 1 -- import necessary libraries

In [34]:
import importlib

import numpy as np
import pandas as pd
import seaborn as sns
import sklearn.preprocessing as skpp

import dask.dataframe as dd

import os # os.path
import datetime

%matplotlib inline
from matplotlib import pyplot as plt
from numpy import array
from tensorflow.keras.activations import relu, softsign
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, TimeDistributed, Dropout
from tensorflow.keras.models import Sequential
import tensorflow as tf

%load_ext tensorboard

import foresight.backtesting
import foresight.data_functions as fx_df
import foresight.model
import foresight.util as fxu

importlib.reload(fx_df)
importlib.reload(fxu)
importlib.reload(foresight.model)
importlib.reload(foresight.backtesting)

from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
mixed_precision.set_policy(policy)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## 2.1 Specify the path of the source data

In [35]:
# src = '/var/local/foresight/timeseries/EURGBP-2016-01.pp1.xz'
src = "/var/local/foresight/timeseries/EURGBP-2017-1-6.csv"

### 2.1.1 Setup model parameters

In [63]:
scaler_type = "MinMaxScaler"  # [MinMaxScaler, None]
sample_time = "5T"  # T=minutes
seq_len = 12  # 1 hours
n_vars = 1
num_outs = 1  # number of future outputs to forecast
Transform = "LogDiff"  # [Diff, LogDiff, None]
model_name = '0004a_6-299_LSTM_5minData_1h_mse'
models_base_path = '/var/local/foresight/models/Keras_Models/'
model_path = models_base_path + model_name

model_LSTM_units = 288

### 2.1.2 Setup training parameters

In [64]:
training_batch_size = 64
training_epochs = 5000
training_dropoff = 0.0
training_loss = 'mse' # ['mae', 'mse', 'mape']
training_optimizer = 'adam' # ['adam', 'nadam', ...] (see tf docs)

# 3 Prepare Data

## 3.1 Import the data

1. Load the data into a dataframe
2. Specify the column names
3. Convert the date field into the correct datetime64 object

In [65]:
data_raw = fx_df.GetTickdataDataframe(src, date_format_string='%m/%d/%Y %H:%M:%S.%f')

## 3.2 Clean up the data

In [66]:
if not (fxu.VarExists(sample_time)):
    raise RuntimeError("'sample_time' must be defined")

data = fx_df.clean_data(
    data_raw,
    remove_duplicates=True,
    sample_frequency=sample_time,
    sample_type="nearest",
    remove_weekends=True,
)

## 3.3 Create transformer object

In [67]:
data_txr = fx_df.Data_Transformer(Transform, 2, scaler_type)

## 3.4 Extract data as an np.ndarray

In [68]:
data_arr = data["bid"].to_numpy()

# 4 Prepare Keras model

In [69]:
def BuildModel():
    if not (fxu.VarExists(seq_len)):
        raise RuntimeError("'seq_len' must be defined")

    if not (fxu.VarExists(n_vars)):
        raise RuntimeError("'n_vars' must be defined")
    
    # TODO: fix this somehow
    output_columns = [0]

    Activation_Function = "tanh"  # ['tanh', 'softsign', 'relu', 'elu', 'LeakyReLU']

    _metrics = [tf.keras.metrics.MeanAbsoluteError(name='mae'),
                tf.keras.metrics.MeanAbsolutePercentageError(name='mape'), 
                tf.keras.metrics.MeanSquaredError(name='mse'), 
#                tf.keras.metrics.RootMeanSquaredError(name='rmse'), 
#                tf.keras.metrics.LogCoshError(name='logcosh'), 
#                tf.keras.metrics.MeanSquaredLogarithmicError(name='msle')
               ]
    
    LSTM_model = Sequential()

#    LSTM_model.add(LSTM(100, activation=Activation_Function, return_sequences=True, input_shape=(seq_len, n_vars), dropout=training_dropoff))
    LSTM_model.add(LSTM(model_LSTM_units, activation=Activation_Function, return_sequences=True, input_shape=(seq_len, n_vars), dropout=0))
    LSTM_model.add(LSTM(model_LSTM_units, activation=Activation_Function, return_sequences=True, dropout=training_dropoff))
    LSTM_model.add(LSTM(model_LSTM_units, activation=Activation_Function, return_sequences=True, dropout=training_dropoff))
    LSTM_model.add(LSTM(model_LSTM_units, activation=Activation_Function, return_sequences=True, dropout=training_dropoff))
    LSTM_model.add(LSTM(model_LSTM_units, activation=Activation_Function, return_sequences=True, dropout=training_dropoff))
    LSTM_model.add(LSTM(model_LSTM_units, activation=Activation_Function, return_sequences=False, dropout=training_dropoff))
    if training_dropoff:
        LSTM_model.add(Dropout(training_dropoff))
    LSTM_model.add(Dense(len(output_columns)))
    # model.compile(loss= 'mae' , optimizer= 'nadam' )
    LSTM_model.compile(loss= training_loss , optimizer= training_optimizer, metrics = _metrics ) # 'adam'
    # model.build()
    print(LSTM_model.summary())
    
    return LSTM_model

# 5 Create foresight.Model object

In [85]:
fx_model = foresight.model.Model(
    model=BuildModel(),
    data=data_arr,
    data_freq=pd.Timedelta(sample_time),
    seq_len=seq_len,
    scaler=None,
    forecast_horizon=1,
    data_transform=data_txr,
    stationary_transform="LogDiff",
    max_training_data_factor=2.0,
)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_19 (LSTM)               (None, 12, 288)           334080    
_________________________________________________________________
lstm_20 (LSTM)               (None, 12, 288)           664704    
_________________________________________________________________
lstm_21 (LSTM)               (None, 12, 288)           664704    
_________________________________________________________________
lstm_22 (LSTM)               (None, 12, 288)           664704    
_________________________________________________________________
lstm_23 (LSTM)               (None, 12, 288)           664704    
_________________________________________________________________
lstm_24 (LSTM)               (None, 288)               664704    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

# 6 Fit timeseries Model

## 6.1 Setup callbacks

Callbacks are used here for 

- saving checkpoint info to disk to allow resuming the training of a model if it becomes interrupted
- saving tensorboard log information for visualization

### 6.1.1 Parameters

In [86]:
# Checkpoint parameters
checkpoint_file = model_path + '/checkpoints/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_file)
cp_freq_in_epochs = 10

# Tensorboard Parameters
tb_path = model_path + '/logs'
logdir = os.path.join(tb_path, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

### 6.1.2 Create Checkpoint Callback

In [87]:
cp_approx_batches_per_epoch = (data_arr.shape[0] - seq_len)/training_batch_size
cp_save_freq = int(cp_approx_batches_per_epoch * cp_freq_in_epochs)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_file,
    save_weights_only=True,
    save_freq=cp_save_freq,
)

### 6.1.3 Create Tensorboard Callback

In [88]:
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=10)

## 6.2 Setup model directory and load checkpoint data, if available

In [89]:
if not os.path.exists(checkpoint_dir):
    from pathlib import Path
    Path(checkpoint_dir).mkdir(parents=True, exist_ok=True)
    
if os.path.isfile(checkpoint_file + '.index'):
    fx_model._model.load_weights(checkpoint_file)
    print('Loaded saved weights from checkpoint file')
else:
    fx_model._model.save_weights(checkpoint_file)
    print('Initialized new checkpoint file')


Loaded saved weights from checkpoint file


## 6.3 Fit the model

## 6.3.1 Start tensorboard

In [90]:
#%tensorboard --logdir var/local/foresight/models/Keras_Models/0001_6-LSTM_5minData_3h/logs
%tensorboard --logdir $tb_path --bind_all
#from tensorboard import notebook
#notebook.list()
#notebook.display(port=6006)

In [76]:
_ = fx_model.Fit(
    batch_size=training_batch_size,
    epochs=training_epochs,
    train_frac=5 / 5,
    valid_frac=0 / 3,
    verbose=1,
    validate_model=False,
    print_test_stat=False,
    callbacks=[cp_callback, tb_callback],
#    callbacks=[cp_callback],
)

Number of training samples:  8314
Number of test samples:  0
Number of validation samples:  8314
Epoch 1/5000
130/130 [==============================] - 4s 28ms/step - loss: 0.122300 - mae: 0.268004 - mape: 269.950226 - mse: 0.122299
Epoch 2/5000
130/130 [==============================] - 3s 23ms/step - loss: 0.122045 - mae: 0.267362 - mape: 193.790695 - mse: 0.122049
Epoch 3/5000
130/130 [==============================] - 3s 23ms/step - loss: 0.122008 - mae: 0.267392 - mape: 223.840210 - mse: 0.122007
Epoch 4/5000
130/130 [==============================] - 3s 23ms/step - loss: 0.121985 - mae: 0.267289 - mape: 168.978790 - mse: 0.121981
Epoch 5/5000
130/130 [==============================] - 3s 23ms/step - loss: 0.122024 - mae: 0.267362 - mape: 156.495026 - mse: 0.122027
Epoch 6/5000
130/130 [==============================] - 3s 23ms/step - loss: 0.122060 - mae: 0.267609 - mape: 223.854004 - mse: 0.122056
Epoch 7/5000
130/130 [==============================] - 3s 23ms/step - loss: 0.12

KeyboardInterrupt: 

In [77]:
#fx_model._model.save(models_path + model_name)

# 6 Backtest the model

In [83]:
importlib.reload(foresight.backtesting)
importlib.reload(fx_df)
print(type(fx_model))
backtester = foresight.backtesting.Backtester(
    model=fx_model,
    retraining_freq=pd.Timedelta("1W"),
    trading_rules={
        "trade_size": 1_000,
        "stop_loss": 0.00025,
        "take_profit": 0.00025,
        "min_change": 0.00005,
        "leverage": 1,
    },
    initial_money=1_000,
)

<class 'foresight.model.Model'>
Restarting every 2016 datapoints
New backtester created!!


In [79]:
bt_src = "/var/local/foresight/timeseries/EURGBP-2017-7-13.csv"
forecast_data = fx_df.GetTickdataDataframe(bt_src, date_format_string='%m/%d/%Y %H:%M:%S.%f')

In [92]:
backtester.Reset_Model()
bids, bid_hats = backtester.Backtest(
    forecast_data,
    initial_retraining=1,
    retrain_batch_size=4 * training_batch_size,
    retrain_epochs=25,
    retrain_verbose=True,
)

Retraining every 2016 points
Number of training samples:  14362
Number of test samples:  0
Number of validation samples:  14362
Epoch 1/25
225/225 [==============================] - 5s 23ms/step - loss: 0.056572 - mae: 0.136316 - mape: 924.021729 - mse: 0.056573
Epoch 2/25
225/225 [==============================] - 5s 23ms/step - loss: 0.034996 - mae: 0.107451 - mape: 1042.703857 - mse: 0.034998
Epoch 3/25
225/225 [==============================] - 5s 23ms/step - loss: 0.016493 - mae: 0.071767 - mape: 768.644531 - mse: 0.016492
Epoch 4/25
225/225 [==============================] - 5s 23ms/step - loss: 0.010167 - mae: 0.047306 - mape: 555.146179 - mse: 0.010166
Epoch 5/25
225/225 [==============================] - 5s 23ms/step - loss: 0.004845 - mae: 0.032822 - mape: 402.950226 - mse: 0.004846
Epoch 6/25
225/225 [==============================] - 5s 23ms/step - loss: 0.001908 - mae: 0.023217 - mape: 315.071411 - mse: 0.001908
Epoch 7/25
225/225 [==============================] - 5s 23ms